# UB DataScience: Complex networks
## Final delivery: Barcelona bus network



<br>
### David Solans & Gerard Martinez

In [9]:
# Required Python imports
import pandas as pd
import numpy as np
import networkx as nx
import pickle
import geopandas as gpd
import matplotlib.pyplot as plt
plt.style.use('seaborn')

## Required for data scrapping
import urllib.request
from bs4 import BeautifulSoup
from selenium import webdriver

%matplotlib inline

### Obtaining the data

### Data exploration

### Macroscale
* Average shortest path
* Comparación 
* Radio
* Diametro
* Node centralities
* Degree
* Betweeness
* Closeness
* Periphery + plot

### Microscale

* Clustering (?)



### Random graphs comparison
* Connected Watts Strotgaz
* Erdos Renyi
* Barabsi Albert

In [ ]:
### Mesoscale
* Community detection
** Cliques
** Label propagation
** Fluid communities
** Louvain
** Girvan Newman

### Dynamic models
* Dynamic modelsReplicar semana 6 -> Simular infección (modelo dinámico) en dos paradas (mayor centrality, menor centrality). Comparar resultados tras 1000 iteraciones (10 ejecuciones para sacar medias).
* Voter model
* Diffusion model